# Example Bokeh Application Embedded in Jupyter Notebook

Hello, world / Iris data set example to illustrate embedding of Bokeh applications within a Jupyter notebook.

Note that this example will only run with the latest version of [bokeh (version 0.12.5)](http://bokeh.pydata.org/en/latest/#).

In [1]:
import pandas as pd
from bokeh.layouts import row, widgetbox
from bokeh.models import Select
from bokeh.charts import Histogram
from bokeh.io import show
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

## Load the Iris Data Set

In [3]:
iris_df = pd.read_csv("data/iris.data", 
    names=["Sepal Length", "Sepal Width", "Petal Length", "Petal Width", "Species"])
feature_names = iris_df.columns[0:-1].values.tolist()
iris_df.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [10]:
# Load reference dict:
import json
import os
source = '/nas/volume1/2photon/projects'
experiment = 'gratings_phaseMod'
session = '20170825_CE055'
acquisition = 'FOV1_planar'
functional_dir = 'functional_test'

acquisition_dir = os.path.join(source, experiment, session, acquisition)

refdict_fn = 'reference_%s2.json' % functional_dir
with open(os.path.join(acquisition_dir, refdict_fn), 'r') as fr:
    ref = json.load(fr)
    
ref

{u'acquisition': u'FOV1_planar',
 u'acquisition_base_dir': u'/nas/volume1/2photon/projects/gratings_phaseMod/20170825_CE055/FOV1_planar',
 u'data_dir': u'/nas/volume1/2photon/projects/gratings_phaseMod/20170825_CE055/FOV1_planar/functional_test/DATA',
 u'experiment': u'gratings_phaseMod',
 u'functional': u'functional_test',
 u'mcparams_path': u'/nas/volume1/2photon/projects/gratings_phaseMod/20170825_CE055/FOV1_planar/functional_test/DATA/mcparams.mat',
 u'nchannels': 2,
 u'ntiffs': 1,
 u'roi_id': u'blobs_DoG',
 u'roi_method': u'pyblob2D',
 u'roiparams_path': u'/nas/volume1/2photon/projects/gratings_phaseMod/20170825_CE055/FOV1_planar/ROIs/blobs_DoG/roiparams.mat',
 u'session': u'20170825_CE055',
 u'signal_channel': 1,
 u'simeta_path': u'/nas/volume1/2photon/projects/gratings_phaseMod/20170825_CE055/FOV1_planar/functional_test/DATA/simeta.mat',
 u'slices': 1,
 u'source': u'/nas/volume1/2photon/projects',
 u'trace_dir': u'/nas/volume1/2photon/projects/gratings_phaseMod/20170825_CE055/FO

In [13]:
roi_dir = os.path.join(ref['acquisition_base_dir'], 'ROIs')
roi_names = os.listdir(roi_dir)
print "ROI methods:", roi_names

trace_dir = os.path.join(ref['acquisition_base_dir'], 'Traces')
trace_names = os.listdir(trace_dir)
print "Trace methods:", trace_names


ROI methods: [u'blobs_LoG', u'blobs_DoG']
Trace methods: [u'blobs_DoG']


In [25]:
roi_opts = [str(r) for r in roi_names]
trace_opts = [str(r) for r in trace_names]

if isinstance(ref['slices'], int):
    slice_names = ['Slice001']
else:
    slice_names = ["Slice{:02d}".format(i+1) for i in range(len(ref['slices']))]
    
slice_names

['Slice001']

In [201]:
average_slice_dir = os.path.join(ref['data_dir'], 'Averaged_Slices', 'Channel01')
files = [f for f in os.listdir(average_slice_dir) if '_visible' not in f]
print files

curr_file = files[0]
print curr_file

# roi_slice_dir = os.path.join(roi_dir, ref['roi_id'], 'figures')
# roi_slice_imgs = os.listdir(roi_slice_dir)
# curr_file = roi_slice_imgs[0]


[u'File001']
File001


In [242]:
import scipy.io as spio
import numpy as np

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)

    return _check_keys(data)


def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])

    return dict


def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        elif isinstance(elem,np.ndarray):
            dict[strg] = _tolist(elem)
        else:
            dict[strg] = elem

    return dict


def _tolist(ndarray):
    '''
    A recursive function which constructs lists from cellarrays 
    (which are loaded as numpy ndarrays), recursing into the elements
    if they contain matobjects.
    '''
    elem_list = []
    for sub_elem in ndarray:
        if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
            elem_list.append(_todict(sub_elem))
        elif isinstance(sub_elem,np.ndarray):
            elem_list.append(_tolist(sub_elem))
        else:
            elem_list.append(sub_elem)

    return elem_list


In [251]:
roiparams_path = os.path.join(ref['acquisition_base_dir'], 'ROIs', current_roi_name, 'roiparams.mat')
roiparams = loadmat(roiparams_path)
#print roiparams['roiparams'].keys()
maskpaths = roiparams['roiparams']['maskpaths']
#print maskpaths
if isinstance(maskpaths, unicode):
    masks = loadmat(maskpaths)
else:
    masks = loadmat(maskpaths[slice_idx])

['nrois', 'maskpaths', 'maskpath3d', 'params', 'roi_info', 'sourcepaths']
/nas/volume1/2photon/projects/gratings_phaseMod/20170825_CE055/FOV1_planar/ROIs/blobs_DoG/masks/20170825_CE055_FOV1_planar_Slice01_Channel01_masks.mat


In [247]:

masks = loadmat(maskpaths)


['__version__', '__header__', 'masks', '__globals__']

In [249]:
masks['masks'].shape

(512, 512, 247)

In [202]:
from bokeh.plotting import figure
import tifffile as tf
import cv2

import matplotlib.pylab as plt
%matplotlib notebook

from skimage import img_as_uint

## Create Embedded Application

In [299]:
# Create the Document Application
def modify_doc(doc):
    
    # Create the main plot
    def create_figure():
        
        current_slice_dir = os.path.join(average_slice_dir, curr_file)
        slice_fns = os.listdir(current_slice_dir)
        slice_fns = [s for s in slice_fns if s.endswith('.tif')]
        
        current_roi_name = roi_name.value
        current_slice_name = slice_name.value
#         p = Histogram(iris_df, current_feature_name, title=current_feature_name, color='Species', 
#             bins=20, legend='top_right', width=600, height=400)

        slice_idx = slice_names.index(current_slice_name)
        
        # GET masks:
        roiparams_path = os.path.join(ref['acquisition_base_dir'], 'ROIs', current_roi_name, 'roiparams.mat')
        roiparams = loadmat(roiparams_path)
        #print roiparams['roiparams'].keys()
        maskpaths = roiparams['roiparams']['maskpaths']
        #print maskpaths
        if isinstance(maskpaths, unicode):
            masks = loadmat(maskpaths)
        else:
            masks = loadmat(maskpaths[slice_idx])
        curr_mask = masks['masks'][:,:,100]
        curr_mask = curr_mask[::-1]
        xs=[]; ys=[]
        for x in range(curr_mask.shape[1]):
            for y in range(curr_mask.shape[0]):
                if curr_mask[y,x]==1:
                    xs.append(x)
                    ys.append(y)
        xs = [float(x)/curr_mask.shape[0] for x in xs]
        ys = [float(y)/curr_mask.shape[1] for y in ys]
        print xs
                    
        #curr_mask = curr_mask[::-1]
    
        # GET average slice image:
        tiff_path = os.path.join(current_slice_dir, slice_fns[slice_idx])
        
        with tf.TiffFile(tiff_path) as tif:
            img = tif.asarray()
        
        # reverse y axis manualy
        img = img[::-1]
        # img = cv2.imread(tiff_path)
         
        fov = np.zeros((img.shape[0], img.shape[1], 4))
        fov[:,:,1] = img
        fov[:,:,0] = curr_mask
        fov[:,:,3] = np.ones((img.shape[0], img.shape[1]))*255
        
    
        #p = figure(plot_width=600, plot_height=400) #, x_range=[0, img.shape[0]], y_range=[0, img.shape[1]])
        p = figure(plot_width=600, plot_height=400, x_range=(0,1), y_range=(0,1))
        #p.image_url(image=[tiff_path], x=[0], y=[0], dw=None, dh=None, dilate=True)
        
        p.image([img], 0, 0, 1, 1, dilate=True) #], x=[0], y=[0], dw=None, dh=None, dilate=True)
        #p.image([curr_mask], 0, 0, 1, 1, dilate=True, alpha=0.5)
        #p.image_rgba([fov], 0, 0, 1, 1) #, dilate=True) #], x=[0], y=[0], dw=None, dh=None, dilate=True)
        p.patch(xs, ys, alpha=1, line_width=2)
            
        # Set the x axis label
        p.xaxis.axis_label = current_slice_name

        # Set the y axis label
        p.yaxis.axis_label = ''
        return p

    # Update the plot
    def update(attr, old, new):
        layout.children[1] = create_figure()
    
    # Controls
    roi_name = Select(title="ROI method:", options=roi_opts, value=roi_opts[0])
    roi_name.on_change('value', update)
    
    slice_name = Select(title="Slice:", options=slice_names, value=slice_names[0])
    slice_name.on_change('value', update)
    controls = widgetbox([slice_name], width=200)
    p = create_figure()
    layout = row(controls, p)
    doc.add_root(layout)

# Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [300]:
# Create the Document
# Not strictly necessary, but helps w/ debugging
doc = app.create_document()

[0.912109375, 0.912109375, 0.912109375, 0.9140625, 0.9140625, 0.9140625, 0.9140625, 0.9140625, 0.9140625, 0.9140625, 0.916015625, 0.916015625, 0.916015625, 0.916015625, 0.916015625, 0.916015625, 0.916015625, 0.916015625, 0.916015625, 0.916015625, 0.916015625, 0.91796875, 0.91796875, 0.91796875, 0.91796875, 0.91796875, 0.91796875, 0.91796875, 0.91796875, 0.91796875, 0.91796875, 0.91796875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.919921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.921875, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.923828125, 0.92578125, 0.92578125, 0.92578125, 0.92578125, 0.92578125, 0.92578125, 0.92578125, 0.92578125, 0.92578125, 0.9257

In [301]:
# Show the application
# Make sure the URL matches your Jupyter instance
show(app, notebook_url="localhost:8888")


In [284]:
show(p)  # open a browser
